- 다음모임때 준비할 것 
  . 본인 회사 주식, 2018년부터 지금까지 데이터 확보하기 (교재는 2021년까지임.)
    : 증권회사 또는 네이버 등 본인이 편한 방법으로
  . 잘 얻어졌다면 업데이트 된 데이터로 프로그램 돌려보기 (옵션임)

- date,
- open,
- high
- low,
- close,
- volume,
- per,
- pbr,
- roe,

- open_lastclose_ratio,
- high_close_ratio,
- low_close_ratio,
- diffratio,
- volume_lastvolume_ratio,

- close_ma5_ratio,
- volume_ma5_ratio,
- close_ma10_ratio,
- volume_ma10_ratio,
- close_ma20_ratio,
- volume_ma20_ratio,
- close_ma60_ratio,
- volume_ma60_ratio,
- close_ma120_ratio,
- volume_ma120_ratio,

ind,ind_diff,ind_ma5,ind_ma10,ind_ma20,ind_ma60,ind_ma120,
inst,inst_diff,inst_ma5,inst_ma10,inst_ma20,inst_ma60,inst_ma120,
foreign,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120

In [76]:
import FinanceDataReader as fdr
from pykrx import stock
# import pandas_datareader as pdr
import pandas_datareader.data as web
import pandas as pd

import matplotlib.pyplot as plt

In [77]:
# 데이터 수집 및 머지

# df = web.DataReader('022100', 'naver', start='2018-01-01', end='2023-12-31')
df1 = fdr.DataReader('022100', '2018-01-01', '2023-12-31')
df2 = stock.get_market_fundamental("20180101", "20231231", "022100")
df3 = stock.get_market_trading_value_by_date("20180101", "20231231", "022100")

df_temp = df1.merge(df2, left_index=True, right_index=True, how='inner')
df = df_temp.merge(df3, left_index=True, right_index=True, how='inner')

df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,기관합계,기타법인,개인,외국인합계,전체
Date,,,,,,,,,,,,,,,,,
2018-01-02,8420,8480,8270,8460,1176650,0.013174,2461,34.81,3.44,243,0.59,50,-477733190,-134870360,108029810,504573740,0
2018-01-03,8460,8710,8420,8500,1678788,0.004728,2461,34.98,3.45,243,0.59,50,-679358880,-24857520,2023406240,-1319189840,0
2018-01-04,8580,8590,8210,8220,1546841,-0.032941,2461,33.83,3.34,243,0.61,50,-1553891240,75680000,1758113290,-279902050,0
2018-01-05,8230,8360,8230,8300,1035357,0.009732,2461,34.16,3.37,243,0.60,50,-879541070,-184147630,319572730,744115970,0
2018-01-08,8310,8320,8050,8100,1446225,-0.024096,2461,33.33,3.29,243,0.62,50,-1990607460,-99806700,1921453340,168960820,0


In [78]:
# roe 구하기
df['ROE'] = df['PBR'] / df['PER']
# 컬럼명 정리
df.rename(columns={'개인': 'ind'}, inplace=True)
df.rename(columns={'기관합계': 'inst'}, inplace=True)
df.rename(columns={'외국인합계': 'foreign'}, inplace=True)

# 컬럼 이름을 소문자로 변환
df.columns = df.columns.str.lower()

# 컬럼 삭제
df = df.drop('기타법인', axis=1)
df = df.drop('전체', axis=1)

df.head()

,open,high,low,close,volume,change,bps,per,pbr,eps,div,dps,inst,ind,foreign,roe
Date,,,,,,,,,,,,,,,,
2018-01-02,8420,8480,8270,8460,1176650,0.013174,2461,34.81,3.44,243,0.59,50,-477733190,108029810,504573740,0.098822
2018-01-03,8460,8710,8420,8500,1678788,0.004728,2461,34.98,3.45,243,0.59,50,-679358880,2023406240,-1319189840,0.098628
2018-01-04,8580,8590,8210,8220,1546841,-0.032941,2461,33.83,3.34,243,0.61,50,-1553891240,1758113290,-279902050,0.098729
2018-01-05,8230,8360,8230,8300,1035357,0.009732,2461,34.16,3.37,243,0.60,50,-879541070,319572730,744115970,0.098653
2018-01-08,8310,8320,8050,8100,1446225,-0.024096,2461,33.33,3.29,243,0.62,50,-1990607460,1921453340,168960820,0.098710


In [79]:
# 모든 열의 데이터 타입을 숫자형으로 변환
# df = df.apply(pd.to_numeric, errors='coerce')
df.dtypes

open         int64
high         int64
low          int64
close        int64
volume       int64
change     float64
bps          int32
per        float64
pbr        float32
eps          int32
div        float32
dps          int32
inst         int64
ind          int64
foreign      int64
roe        float64
dtype: object

In [80]:
# 비율 파생변수 last 접두사는 전일을 의미함
df['open_lastclose_ratio'] = df['open'] / df['close'].shift(1)
df['high_close_ratio'] = df['high'] / df['close']
df['low_close_ratio'] = df['low'] / df['close']
df['diffratio'] = df['open'] / df['close']
df['volume_lastvolume_ratio'] = df['volume'] / df['volume'].shift(1)


In [81]:
# 종가 이동평균으로 나눈 현재 종가 비율
df['close_ma5_ratio'] = df['close'] / df['close'].rolling(window=5).mean()
df['close_ma10_ratio'] = df['close'] / df['close'].rolling(window=10).mean()
df['close_ma20_ratio'] = df['close'] / df['close'].rolling(window=20).mean()
df['close_ma60_ratio'] = df['close'] / df['close'].rolling(window=60).mean()
df['close_ma120_ratio'] = df['close'] / df['close'].rolling(window=120).mean()

# 거래량 이동평균으로 나눈 현재 거래량 비율
df['volume_ma5_ratio'] = df['volume'] / df['volume'].rolling(window=5).mean()
df['volume_ma10_ratio'] = df['volume'] / df['volume'].rolling(window=10).mean()
df['volume_ma20_ratio'] = df['volume'] / df['volume'].rolling(window=20).mean()
df['volume_ma60_ratio'] = df['volume'] / df['volume'].rolling(window=60).mean()
df['volume_ma120_ratio'] = df['volume'] / df['volume'].rolling(window=120).mean()
df.head()

,open,high,low,close,volume,change,bps,per,pbr,eps,...,close_ma5_ratio,close_ma10_ratio,close_ma20_ratio,close_ma60_ratio,close_ma120_ratio,volume_ma5_ratio,volume_ma10_ratio,volume_ma20_ratio,volume_ma60_ratio,volume_ma120_ratio
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,8420,8480,8270,8460,1176650,0.013174,2461,34.81,3.44,243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,8460,8710,8420,8500,1678788,0.004728,2461,34.98,3.45,243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,8580,8590,8210,8220,1546841,-0.032941,2461,33.83,3.34,243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,8230,8360,8230,8300,1035357,0.009732,2461,34.16,3.37,243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,8310,8320,8050,8100,1446225,-0.024096,2461,33.33,3.29,243,...,0.974026,NaN,NaN,NaN,NaN,1.050446,NaN,NaN,NaN,NaN


In [82]:
# ind,ind_diff,ind_ma5,ind_ma10,ind_ma20,ind_ma60,ind_ma120,
# inst,inst_diff,inst_ma5,inst_ma10,inst_ma20,inst_ma60,inst_ma120,
# foreign,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120

# 개인 거래량 이동평균
df['ind_diff'] = df['ind'] - df['ind'].shift(1)
df['ind_ma5'] = df['ind'].rolling(window=5).mean()
df['ind_ma10'] = df['ind'].rolling(window=10).mean()
df['ind_ma20'] = df['ind'].rolling(window=20).mean()
df['ind_ma60'] = df['ind'].rolling(window=60).mean()
df['ind_ma120'] = df['ind'].rolling(window=120).mean()

# 기관 거래량 이동평균
df['inst_diff'] = df['inst'] - df['inst'].shift(1)
df['inst_ma5'] = df['inst'].rolling(window=5).mean()
df['inst_ma10'] = df['inst'].rolling(window=10).mean()
df['inst_ma20'] = df['inst'].rolling(window=20).mean()
df['inst_ma60'] = df['inst'].rolling(window=60).mean()
df['inst_ma120'] = df['inst'].rolling(window=120).mean()

# 외국인 거래량 이동평균
df['foreign_diff'] = df['inst'] - df['inst'].shift(1)
df['foreign_ma5'] = df['inst'].rolling(window=5).mean()
df['foreign_ma10'] = df['inst'].rolling(window=10).mean()
df['foreign_ma20'] = df['inst'].rolling(window=20).mean()
df['foreign_ma60'] = df['inst'].rolling(window=60).mean()
df['foreign_ma120'] = df['inst'].rolling(window=120).mean()
df.head()

,open,high,low,close,volume,change,bps,per,pbr,eps,...,inst_ma10,inst_ma20,inst_ma60,inst_ma120,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,8420,8480,8270,8460,1176650,0.013174,2461,34.81,3.44,243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,8460,8710,8420,8500,1678788,0.004728,2461,34.98,3.45,243,...,NaN,NaN,NaN,NaN,-2.016257e+08,NaN,NaN,NaN,NaN,NaN
2018-01-04,8580,8590,8210,8220,1546841,-0.032941,2461,33.83,3.34,243,...,NaN,NaN,NaN,NaN,-8.745324e+08,NaN,NaN,NaN,NaN,NaN
2018-01-05,8230,8360,8230,8300,1035357,0.009732,2461,34.16,3.37,243,...,NaN,NaN,NaN,NaN,6.743502e+08,NaN,NaN,NaN,NaN,NaN
2018-01-08,8310,8320,8050,8100,1446225,-0.024096,2461,33.33,3.29,243,...,NaN,NaN,NaN,NaN,-1.111066e+09,-1.116226e+09,NaN,NaN,NaN,NaN


In [83]:
# 결측값을 포함하는 행 제거
df = df.dropna()
df.head()


,open,high,low,close,volume,change,bps,per,pbr,eps,...,inst_ma10,inst_ma20,inst_ma60,inst_ma120,foreign_diff,foreign_ma5,foreign_ma10,foreign_ma20,foreign_ma60,foreign_ma120
Date,,,,,,,,,,,,,,,,,,,,,
2018-06-28,6730,6960,6720,6820,515279,0.017910,2629,24.71,2.59,276,...,9320385.0,-91327080.0,-2.844751e+08,-1.284799e+08,248914010.0,47762214.0,9320385.0,-91327080.0,-2.844751e+08,-1.284799e+08
2018-06-29,6980,6980,6680,6810,375970,-0.001466,2629,24.67,2.59,276,...,-1670024.0,-82405349.5,-2.887560e+08,-1.254044e+08,-177181110.0,-6107034.0,-1670024.0,-82405349.5,-2.887560e+08,-1.254044e+08
2018-07-02,6860,6860,6600,6600,422635,-0.030837,2629,23.91,2.51,276,...,36786414.0,-82209951.5,-2.875146e+08,-1.196644e+08,118113050.0,-30578322.0,36786414.0,-82209951.5,-2.875146e+08,-1.196644e+08
2018-07-03,6610,6810,6610,6790,472316,0.028788,2629,24.60,2.58,276,...,104167525.0,-51970129.5,-2.728167e+08,-1.014171e+08,626350250.0,84930362.0,104167525.0,-51970129.5,-2.728167e+08,-1.014171e+08
2018-07-04,6830,7140,6820,7060,1172483,0.039764,2629,25.58,2.69,276,...,67369718.0,-37172143.0,-2.695318e+08,-9.314909e+07,-523170120.0,143535578.0,67369718.0,-37172143.0,-2.695318e+08,-9.314909e+07


In [84]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'change', 'bps', 'per', 'pbr',
       'eps', 'div', 'dps', 'inst', 'ind', 'foreign', 'roe',
       'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
       'diffratio', 'volume_lastvolume_ratio', 'close_ma5_ratio',
       'close_ma10_ratio', 'close_ma20_ratio', 'close_ma60_ratio',
       'close_ma120_ratio', 'volume_ma5_ratio', 'volume_ma10_ratio',
       'volume_ma20_ratio', 'volume_ma60_ratio', 'volume_ma120_ratio',
       'ind_diff', 'ind_ma5', 'ind_ma10', 'ind_ma20', 'ind_ma60', 'ind_ma120',
       'inst_diff', 'inst_ma5', 'inst_ma10', 'inst_ma20', 'inst_ma60',
       'inst_ma120', 'foreign_diff', 'foreign_ma5', 'foreign_ma10',
       'foreign_ma20', 'foreign_ma60', 'foreign_ma120'],
      dtype='object')

In [85]:
# 데이터프레임을 CSV 파일로 저장
df.to_csv('022100_poscodx.csv')